In [1]:
from src.scraping.google_scraping.review_data_saver import ReviewDataSaver
from src.scraping.google_scraping.review_extractor import ReviewExtractor
from src.scraping.google_scraping.review_loader import ReviewLoader
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.scraping.browser_managment.browser_manager import BrowserManager  # Ensure this is the updated BrowserManager
from src.scraping.browser_managment.parallel_browser_manager import ParallelBrowserManager
from selenium.webdriver.common.by import By
import time
from src.scraping.google_scraping import RestaurantListScraper
from src.scraping.google_scraping.data_saver import DataSaver

from src.scraping.google_scraping.review_interaction_handler import ReviewInteractionHandler
city='Zagreb'
query=f'Restorani {city}'


browser_manager = BrowserManager()
review_handler = ReviewInteractionHandler(browser_manager.driver)
browser_manager.navigate_to_url(f"https://www.google.hr/search?q={query}")
review_handler.accept_cookies()

The chromedriver version (121.0.6167.85) detected in PATH at C:\Users\ivan.zeljeznjak\Desktop\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (122.0.6261.112); currently, chromedriver 122.0.6261.111 is recommended for chrome 122.*, so it is advised to delete the driver in PATH and retry


In [2]:

review_handler.navigate_to_restaurant_list()


scraper = RestaurantListScraper(browser_manager.driver)
# Assume driver has navigated to the restaurant list page
df_restaurants = scraper.scrape_restaurants()

data_saver = DataSaver(df_restaurants, city)
data_saver.save_to_csv()
time.sleep(5)
browser_manager.close_browser()

No new content loaded. End of list reached.
Saved data to C:\Users\ivan.zeljeznjak\DataspellProjects\RedditSentimentAnalysis\data\Zagreb\restaurant_list.csv


In [3]:
df_restaurants


,Restaurant Name
0,Health n joy
1,Restoran Lido
2,Stella bar & Vip restoran
3,Restoran HAS
4,superplanty
...,...
208,Gajbica
209,Restoran Uspinjača
210,Restoran Zinfandel's
211,Boban caffe & restoran


In [4]:
import pandas as pd

# Define the path to the CSV file
file_path = r'C:\Users\ivan.zeljeznjak\DataspellProjects\RedditSentimentAnalysis\data\Zagreb\restaurant_list.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df


,Restaurant Name,Last scraping attempt,Last scraping attempt date
0,Esplanade Zagreb Hotel,NaN,NaN
1,Fisherija,NaN,NaN
2,Korica,NaN,NaN
3,Health n joy,NaN,NaN
4,Restoran Lido,NaN,NaN
...,...,...,...
211,Gajbica,NaN,NaN
212,Restoran Uspinjača,NaN,NaN
213,Restoran Zinfandel's,NaN,NaN
214,Boban caffe & restoran,NaN,NaN


In [1]:
from datetime import datetime
from src.scraping.google_scraping.review_data_saver import ReviewDataSaver
from src.scraping.google_scraping.review_extractor import ReviewExtractor
from src.scraping.google_scraping.review_loader import ReviewLoader
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.scraping.browser_managment.browser_manager import BrowserManager  # Ensure this is the updated BrowserManager
from src.scraping.browser_managment.parallel_browser_manager import ParallelBrowserManager
from selenium.webdriver.common.by import By
import time
from src.scraping.google_scraping.review_interaction_handler import ReviewInteractionHandler

def perform_search(query):
    browser_manager = BrowserManager()
    review_handler = ReviewInteractionHandler(browser_manager.driver)
    review_loader = ReviewLoader(browser_manager.driver)
    review_extractor = ReviewExtractor(browser_manager.driver)
    attempt_status = 'Failed'  # Default to 'Failed', update to 'Success' if no exceptions occur
    attempt_date = datetime.now().strftime("%Y-%m-%d %H:%M")
    
    try:
        browser_manager.navigate_to_url(f"https://www.google.hr/search?q={query}")
        review_handler.accept_cookies()
        review_handler.navigate_to_reviews()
        review_loader.load_reviews()  # Load all reviews
        review_handler.click_more_buttons()  # Click all "More" buttons
        reviews_df = review_extractor.collect_reviews()  # Adjust based on actual return type/method
        attempt_status = 'Success'

        # Success metrics calculations and prints can remain here or be moved as appropriate
        loaded_reviews = review_loader.total_loaded_reviews
        scraped_reviews = review_extractor.total_scraped_reviews
        ratio = scraped_reviews / loaded_reviews if loaded_reviews > 0 else 0
        percentage = (scraped_reviews / loaded_reviews * 100) if loaded_reviews > 0 else 0

        print(f"Query: {query}")
        print(f"Final count of reviews loaded: {loaded_reviews}, Scraped/Loaded Ratio: {scraped_reviews}/{loaded_reviews} = {ratio:.2f}, Success Percentage: {percentage:.2f}%")
        review_data_saver = ReviewDataSaver(reviews_df, query)
        review_data_saver.save_reviews_to_csv()
    except Exception as e:
        print(f"An error occurred during search for '{query}': {e}")
        attempt_status = 'Failed'
    finally:
        browser_manager.close_browser()

    return query, attempt_status, attempt_date




In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from src.scraping.browser_managment.parallel_browser_manager import ParallelBrowserManager

# Replace 'src.scraping' with your actual module path

city = 'Zagreb'
csv_file_path = f'../data/{city}/restaurant_list.csv'
df = pd.read_csv(csv_file_path, parse_dates=['Last scraping attempt date'])

# Ensure the DataFrame has the necessary columns for tracking scraping attempts
if 'Last scraping attempt' not in df.columns:
    df['Last scraping attempt'] = 'Not attempted'
if 'Last scraping attempt date' not in df.columns:
    df['Last scraping attempt date'] = pd.NaT

# Filter the DataFrame for restaurants to attempt scraping
one_month_ago = datetime.now() - timedelta(days=30)
should_scrape = (
        (df['Last scraping attempt'] == 'Not attempted') |
        (df['Last scraping attempt'] == 'Failed') |
        (df['Last scraping attempt date'].isna()) |
        (df['Last scraping attempt date'] < one_month_ago)
)

# Create queries only for those restaurants that meet the conditions
queries = [row['Restaurant Name'] + ' ' + city for index, row in df[should_scrape].iterrows()]

# Prepare the args_list for the ParallelBrowserManager
parallel_manager = ParallelBrowserManager(df, csv_file_path, city, workers=3)



args_list = [(query,) for query in queries]
results = parallel_manager.perform_parallel_action(perform_search, args_list)

# Function to update the DataFrame with the search results
def update_dataframe_with_results(df, results, city):
    for result in results:
        if result:  # Ensure there's a result before processing
            query, status, date = result
            restaurant_name_with_city = query.rsplit(' ', 1)[0]
            # Remove the city name from the result for matching
            restaurant_name = restaurant_name_with_city.replace(f' {city}', '').strip()
            df.loc[df['Restaurant Name'].str.strip() == restaurant_name, 'Last scraping attempt'] = status
            df.loc[df['Restaurant Name'].str.strip() == restaurant_name, 'Last scraping attempt date'] = date


# Update the DataFrame with the results
update_dataframe_with_results(df, results, city)

# Save the updated DataFrame back to CSV
df.to_csv(csv_file_path, index=False)


In [2]:
#placeholder